# Don't Overfit! II

<a href="https://www.kaggle.com/c/dont-overfit-ii">Link to competition on Kaggle</a>

Inspiration taken from the following Kaggle kernels:
- <a href="https://www.kaggle.com/featureblind/robust-lasso-patches-with-rfe-gs">Robust, Lasso, Patches with RFE & GS</a>